
1. <span class="girk">Preprocess CelebA to $ 32 \times 32 $, downsample scaled</span>

2. <span class="girk">generate_latent_points: $128+ 5*10 \;cat \;codes\; + 5 \;cont \; codes$ {183}: Done</span>
3. <span class="girk">Generator_m<span class="girk">odel</span>: Done 
4. <span class="girk">Discriminator_model: Done</span></span>
5. <span class="girk">Auxilary_model: Done</span>

6. <span class="girk">gan_model</span>
7. <span class="burk">train_info_gan_CelebA</span>
8. <span class="burk">info_gan_evaluate</span>
9. <span class="girk">generate_real_samples, labeles</span>
10. <span class="girk">generate_fake_samples , labels</span>

11. <span class="girk">Build Projector Network</span> 
12. <span class="burk">train the projector network</span>

13. <span class="burk">save the whole model</span>
14. load the saved model and <span class="burk">reuse it in ADMM+GAN program</span> 
    

In [1]:
from keras import layers
from keras import optimizers
from keras.models import Model
from keras.initializers import RandomNormal
import numpy as np
from keras.datasets.mnist import load_data
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot
import os

Using TensorFlow backend.


In [2]:
def define_generator(gen_inp_size):
    init = RandomNormal(stddev = 0.02)
    in_lat = layers.Input(shape = (gen_inp_size,))
    n_nodes = 2*2*366
    gen = layers.Dense(n_nodes, kernel_initializer = init)(in_lat)
    gen = layers.Activation('relu')(gen)
    gen = layers.BatchNormalization()(gen)
    
    gen = layers.Reshape((2,2, 366))(gen)
    
    gen = layers.Conv2DTranspose(256,(4,4), strides=(2,2), padding = 'same', kernel_initializer = init)(gen)
    gen = layers.Activation('relu')(gen)
    gen = layers.BatchNormalization()(gen)
    
    gen = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding = 'same', kernel_initializer = init)(gen)
    gen = layers.Activation('relu')(gen)
    gen = layers.BatchNormalization()(gen)
    
    gen = layers.Conv2DTranspose(64, (4,4), strides=(2,2), padding = 'same', kernel_initializer = init)(gen)
    gen = layers.Activation('relu')(gen)
    gen = layers.BatchNormalization()(gen)
    
    gen = layers.Conv2DTranspose(1, (4,4), strides = (2,2), padding = 'same', kernel_initializer = init)(gen)
    out_layer = layers.Activation('tanh')(gen)
    model = Model(in_lat,out_layer)
    return model

In [3]:
Gen = define_generator(183)
Gen.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 183)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1464)              269376    
_________________________________________________________________
activation_1 (Activation)    (None, 1464)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1464)              5856      
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 2, 366)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 256)         1499392   
_________________________________________________________________
activation_2 (Activation)    (None, 4, 4, 256)         0   

In [3]:
def define_discriminator(n_cat, in_shape= (32,32,1)):
    init = RandomNormal(stddev= 0.02)
    in_img = layers.Input(shape=(in_shape))
    d = layers.Conv2D(64,(5,5), strides = (2,2), padding = 'same', kernel_initializer = init)(in_img)
    d = layers.LeakyReLU(alpha = 0.1)(d)
    
    d = layers.Conv2D(128, (5,5), strides=(2,2), padding = 'same', kernel_initializer = init)(d)
    d = layers.LeakyReLU(alpha = 0.1)(d)
    d = layers.BatchNormalization()(d)
    
    d = layers.Conv2D(256, (5,5), strides=(2,2), padding = 'same', kernel_initializer = init)(d)
    d = layers.LeakyReLU(alpha = 0.1)(d)
    d = layers.BatchNormalization()(d)
    
    d_4096 = layers.Flatten()(d)
    
    d_100 = layers.Dense(100, activation = 'relu')(d_4096)
    out_classifier = layers.Dense(1, activation = 'sigmoid')(d_100)
    
    d_model = Model(in_img, out_classifier)
    d_model.compile(loss ='binary_crossentropy', optimizer = optimizers.Adam(lr=0.0002, beta_1=0.5))
    
    q = layers.Dense(500)(d_4096)
    q = layers.BatchNormalization()(q)
    q = layers.LeakyReLU(alpha=0.1)(q)

    out_codes = layers.Dense(n_cat, activation='softmax')(q)

    q_model = Model(in_img, out_codes)
    return d_model, q_model 

In [ ]:
[d_model,q_model] = define_discriminator(55)
d_model.summary()
q_model.summary()

In [27]:
def info_gan(g_modeld_model, q_model):
    d_model.trainable = False
    d_output = d_model(g_model.output)
    q_output = q_model(g_model.output)
    
    model = Model(g_model.input, [d_output, q_output])

    opt = optimizers.Adam(lr=0.0002, beta_1=0.5)
    # loss function for q model is required, need to learn about training parameters 
    model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt) 
    return model


In [16]:
def make_cat_codes(n_cat,nsamples):
    cat_code = np.random.randint(0,n_cat,nsamples)
    cat_code = to_categorical(cat_code, num_classes = n_cat)
    return cat_code
def make_cont_codes(n_cont_codes, n_samples):
    cont_codes = np.random.uniform(-1,1,size = (n_samples, n_cont_codes))
    return cont_codes

In [17]:
def generate_latent_points(latent_dim, n_cat,  n_cont_codes, n_samples):
    z_latent = np.random.randn(latent_dim*n_samples)
    z_latent = z_latent.reshape(n_samples, latent_dim)
    
    cat_codes_1 = make_cat_codes(n_cat, n_samples)
    cat_codes_2 = make_cat_codes(n_cat, n_samples)
    cat_codes_3 = make_cat_codes(n_cat, n_samples)
    cat_codes_4 = make_cat_codes(n_cat, n_samples)
    cat_codes_5 = make_cat_codes(n_cat, n_samples)
    cont_codes = make_cont_codes(n_cont_codes, n_samples)
    z_input = np.hstack((z_latent, cat_codes_1, cat_codes_2, cat_codes_3, cat_codes_4, cat_codes_5,cont_codes))
    cat_codes = np.hstack((cat_codes_1, cat_codes_2, cat_codes_3, cat_codes_4, cat_codes_5))
    return [z_input, cat_codes]
    
    
    

In [18]:
[z_input, cat_codes] = generate_latent_points(128,10,5,2)
print(z_input[0])

[-0.91449608 -1.60867552 -1.22145304 -0.28034834  0.98569485 -0.76283041
 -1.89962731 -0.25878291 -0.28354671  0.03254996 -0.54362888  0.02492859
  0.70326164 -1.17299303 -0.80526936  0.38828118  0.33141437  0.46465464
  1.25215037  0.36393125  1.3899868   1.31961896 -0.66841591 -0.36417725
 -0.22029485  1.4581492   0.41393907  1.496237   -1.20661929 -1.65389274
  0.01799235 -1.61287137  0.37220587 -1.45846148 -3.32077022  0.41997015
  0.58607667 -0.28097571 -0.47512762 -0.55640195 -0.14002707  0.19984651
  1.36445505 -1.30462522 -1.43957392 -1.34048669 -0.19659868  0.01264574
  0.63395723 -0.12922769  1.09333508  0.12137325  0.09938475  1.7799046
  0.5280385   0.24034489  0.10388196  0.40497953 -0.34427139 -1.98739617
 -1.03163629 -0.24238362  0.61821349  0.13701856  1.07709661 -1.0131991
  0.16867849 -0.78426097  0.57866356 -1.35604041 -0.34696609  0.50667101
 -0.85698494 -0.3701407  -0.29980472 -2.16410441 -1.14499083  0.47257054
  0.55631795 -0.27540916  0.69463523 -0.22100967 -0.7

In [19]:
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = np.expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    print(X.shape)
    return X

def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))
    return X, y

def generate_fake_samples(generator, latent_dim, n_cat, n_samples):
    z_input, _ = generate_latent_points(latent_dim, n_cat, n_samples)
    images = generator.predict(z_input)
    y = np.zeros((n_samples, 1))
    return images, y

In [20]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_cat, n_epochs=100,n_batch=64):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    
    # manually enumerate epochs
    for i in range(20001):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)        
        # update discriminator and q model weights
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        
        # generate "fake" examples
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_cat, half_batch)
        # update discriminator model weights
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        
        # prepare points in latent space as input for the generator
        z_input, cat_codes = generate_latent_points(latent_dim, n_cat, n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # update the g via the d and q error
        _,g_1,g_2 = gan_model.train_on_batch(z_input, [y_gan, cat_codes])
        # summarize loss on this batch
        print('>%d, d[%.3f,%.3f], g[%.3f] q[%.3f]' % (i+1, d_loss1, d_loss2, g_1, g_2))
        # evaluate the model performance every epoch
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, gan_model, latent_dim, n_cat)

In [21]:
def summarize_performance(step, g_model, gan_model, latent_dim, n_cat, n_samples=100):
    X, _ = generate_fake_samples(g_model, latent_dim, n_cat, n_samples)
    X = (X + 1) / 2.0
    
    for i in range(100):
        pyplot.subplot(10, 10, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
        
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(os.path.join(save_dir, filename1))
    pyplot.close()
    
    
    filename2 = 'model_%04d' % (step+1)
    g_model.save(os.path.join(save_dir,filename2))

    filename3 = 'gan_model_%04d' % (step+1)
    gan_model.save(os.path.join(save_dir, filename3))
    
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [22]:
def define_projector(inp):
    inp 
    proj = layers.Dense(1024, activation = 'relu')(inp)
    proj = layers.Dense(512,activation = 'relu')(proj)
    proj = layers.Dense(256,activation = 'relu')(proj)
    out = layers.Dense(183, activation = 'relu')(proj)
    projector = Model(inp,out)
    return projector
    

In [26]:
n_cat = 10
latent_dim = 128
n_cont_codes = 5

d_model, q_model = define_discriminator(n_cat)

gen_input_size = latent_dim + n_cat + n_cont_codes
g_model = define_generator(gen_input_size)

gan_model = info_gan(g_model, d_model, q_model)

dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim, n_cat,n_cont_codes)

ValueError: Dimensions must be equal, but are 1 and 183 for 'model_12/dense_20/MatMul' (op: 'MatMul') with input shapes: [?,1], [183,1464].